In [ ]:
import json
import time
import os
import sys

import ccxt

import pandas as pd
import numpy as np
# https://github.com/ccxt/ccxt/blob/master/examples/py/binance-fetch-ohlcv-pagination.py

In [ ]:
from read_data import get_dir_file_names

In [ ]:
def connect_exchange_kucoin():
    from decouple import AutoConfig
    from decouple import config
    
    config = AutoConfig(' ') # https://github.com/henriquebastos/python-decouple/issues/116
    
    kc_api_key = config('kc_api_key')
    kc_api_secret = config('kc_api_secret')
    kc_password = config('kc_password')
    return ccxt.kucoin({
        'apiKey': kc_api_key,
        'secret': kc_api_secret,
        'password': kc_password,
    })

def connect_exchange_binance():
    from decouple import AutoConfig
    from decouple import config
    
    config = AutoConfig(' ') # https://github.com/henriquebastos/python-decouple/issues/116
    
    kc_api_key = config('api_key')
    kc_api_secret = config('api_secret')

    return ccxt.binance({
        'apiKey': kc_api_key,
        'secret': kc_api_secret
    })

def connect_exchange_bybit():
    from decouple import AutoConfig
    from decouple import config
    
    config = AutoConfig(' ') # https://github.com/henriquebastos/python-decouple/issues/116
    
    api_key = config('bb_api_key')
    api_secret = config('bb_api_secret')

    return ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret
    })

def connect_exchange_coinbase():
    from decouple import AutoConfig
    from decouple import config
    
    config = AutoConfig(' ') # https://github.com/henriquebastos/python-decouple/issues/116
    
    api_key = config('cb_api_key')
    api_secret = config('cb_api_secret')
    
    return ccxt.coinbasepro({
        'apiKey': api_key,
        'secret': api_secret
    })


In [ ]:
# exchanges = [ connect_exchange_binance() , connect_exchange_kucoin() , connect_exchange_bybit(), connect_exchange_coinbase() ]
exchanges = [ connect_exchange_binance() ]
# exchanges = [ connect_exchange_coinbase()]
# exchanges = [ connect_exchange_kucoin()]

In [ ]:
# https://github.com/ccxt/ccxt/blob/master/examples/py/binance-fetch-ohlcv-pagination.py


def fetch_ohlcv_with_retry(exchange, symbol='BTC/USDT', timeframe='1h', since = None):
    all_ohlcvs = []
    
    if since == None:
        exchange_name = exchange.name
        since = exchange.parse8601('2017-01-01T00:00:00Z')
        
        if exchange_name == "Bybit":
            since = exchange.parse8601('2021-01-01T00:00:00Z')
            if timeframe == '1h':
                since = exchange.parse8601('2021-07-01T00:00:00Z')

        if exchange_name == "KuCoin":
            since = exchange.parse8601('2017-09-01T00:00:00Z')
            since = exchange.parse8601('2019-09-01T00:00:00Z')
            
        if exchange_name == "Coinbase Pro":
            since = exchange.parse8601('2020-01-01T00:00:00Z')           

    retries = 0
    retry = True

    while retry:
        try:
            ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
            all_ohlcvs += ohlcvs
            if len(ohlcvs):
                print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', exchange.iso8601(ohlcvs[0][0]))
                since = ohlcvs[-1][0] + 1
            else:
                break
            time.sleep(exchange.rateLimit / 1000) # time.sleep wants seconds
            
        except Exception as e:
            print(type(e).__name__, str(e),symbol, timeframe)
            retries += 1
        except ccxt.RateLimitExceeded as e:
            now = p_exchange.milliseconds()
            datetime = p_exchange.iso8601(now)
            print(datetime, retries, type(e).__name__, str(e))
            p_exchange.sleep(10000)
            retries += 1
            
        if retries>10:
            retry = False

    print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')
    return all_ohlcvs




In [ ]:
def get_candels(exchange, p_symbol='BTC/USDC', p_interval='1m', p_since=None):
    candles = fetch_ohlcv_with_retry(exchange, p_symbol, p_interval, p_since)

    if len(candles):
        candles_df = pd.DataFrame(candles)
        candles_df.columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']
        candles_df["Open time"] = pd.to_numeric(candles_df["Open time"], downcast="integer")
        candles_df["Open"] = pd.to_numeric(candles_df["Open"], downcast="float")
        candles_df["High"] = pd.to_numeric(candles_df["High"], downcast="float")
        candles_df["Low"] = pd.to_numeric(candles_df["Low"], downcast="float")
        candles_df["Close"] = pd.to_numeric(candles_df["Close"], downcast="float")
        candles_df["Volume"] = pd.to_numeric(candles_df["Volume"], downcast="float")
        candles_df.set_index('Open time')
    else:
        candles_df = pd.DataFrame()

    return candles_df


def concat(candle_disk, candels):
    return pd.concat([candle_disk,candels], ignore_index=True).drop_duplicates(['Open time'], keep='last')



In [ ]:
symbols = []

for asset in ['BTC', 'ETH', 'XMR', 'MATIC', 'BNB', 'SOL', 'DOT', 'AVAX', 'ADA','PAXG', 'HNT', 'IOT']:
#for asset in ['HNT']:
    symbols.append(asset + '/USDT')
    #symbols.append(asset + '/USDC')
    if asset != 'BTC':
        symbols.append(asset + '/BTC')
        
timeframes = ['1m', '15m','1h','4h','8h','1d','1w', '1M']
#timeframes = ['1h','1d','1w', '1M']

for exchange in exchanges: 
    exchange_name = exchange.name
    print(exchange_name)
    markets = exchange.load_markets()

    for symbol in symbols:
        if symbol in markets:
            for timeframe in timeframes:
                if timeframe in exchange.timeframes:
                    directory, filename = get_dir_file_names(symbol,timeframe,exchange_name)

                    if not os.path.exists(directory):
                        os.makedirs(directory)

                    since = None
                    if os.path.exists(filename):
                        candle_disk = pd.read_pickle(filename)
                        since = candle_disk["Open time"].iloc[-1]

                    candels = get_candels(exchange, symbol, timeframe, since)

                    if candels.shape[0]>0:
                        if since != None:
                            candels = concat(candle_disk, candels)

                        candels.to_pickle(filename)
                    else:
                        print(f"No data for {exchange_name}, {symbol}, {timeframe}, {since}")
                else:
                    print(f"No timeframe for {exchange_name}, {symbol}, {timeframe}")
        else:
            print(f"{symbol} not in markets in {exchange_name}")

In [ ]:
#since = exchange.parse8601('2021-07-01T00:00:00Z')
#timeframe = '1h'
#ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
#ohlcvs

In [ ]:
#symbol = "BTC/USDT"
#ticker = exchange.fetch_ticker(symbol.upper())
#ticker

In [ ]:
#exchange.name

In [ ]:
#timeframe

In [ ]:
#exchange.currencies.values()

In [ ]:
#symbol = 'BTC/USDT'
#amount = 1.2345678  # amount in base currency BTC
#price = 87654.321  # price in quote currency USDT
#formatted_amount = exchange.amount_to_precision(symbol, amount)
#formatted_price = exchange.price_to_precision(symbol, price)
#print(formatted_amount, formatted_price)

In [ ]:
#symbol="BTC/USDC"
#timeframe="1d"
#since = None
#since = exchange.parse8601('2017-01-01T00:00:00Z')
#ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
#ohlcvs

In [ ]:
#exchange.markets